In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now()}')

Päivitetty 2022-10-25 17:17:55.256970


# Pika-analyysi Exceliin

Frekvenssitaulukot, ristiintaulukoinnit, tilastolliset tunnusluvut ja korrelaatiot Exceliin.

Sinun pitää määritellä, mitkä muuttujista ovat tarpeettomia, kategorisia, mielipideasteikollisia tai dikotomisia. Loput muuttujat tulkitaan määrällisiksi.

Dikotomiset muuttujat ovat muuttujia, joiden arvo on 1 tai puuttuu (puuttuvien tilalla voidaan käyttää myös nollia).

Pika-analyysiä voit käyttää seuraavasti:

* Vaihda avattavan datan tilalle oma data.
* Määritä muuttujien tyypit listoina (tarpeettomat, kategoriset, mielipideasteikot ja dikotomiset). Jätä lista tyhjäksi (pelkät hakasulkeet), jos kyseinen tyyppi puuttuu datasta.
* Koodin suorittamisen jälkeen tulokset avautuvat Exceliin.

In [2]:
import pandas as pd
import xlwings as xw

# Vaihda tähän oma data
df = pd.read_excel('https://taanila.fi/data1.xlsx')
df

,nro,sukup,ikä,perhe,koulutus,palveluv,palkka,johto,työtov,työymp,palkkat,työteht,työterv,lomaosa,kuntosa,hieroja
0,1,1,38,1,1.0,22.0,3587,3,3.0,3,3,3,NaN,NaN,NaN,NaN
1,2,1,29,2,2.0,10.0,2963,1,5.0,2,1,3,NaN,NaN,NaN,NaN
2,3,1,30,1,1.0,7.0,1989,3,4.0,1,1,3,1.0,NaN,NaN,NaN
3,4,1,36,2,1.0,14.0,2144,3,3.0,3,3,3,1.0,NaN,NaN,NaN
4,5,1,24,1,2.0,4.0,2183,2,3.0,2,1,2,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,1,22,1,3.0,0.0,1598,4,4.0,4,3,4,NaN,1.0,1.0,NaN
78,79,1,33,1,1.0,2.0,1638,1,3.0,2,1,2,1.0,NaN,NaN,NaN
79,80,1,27,1,2.0,7.0,2612,3,4.0,3,3,3,1.0,NaN,1.0,NaN
80,81,1,35,2,2.0,16.0,2808,3,4.0,3,3,3,NaN,NaN,NaN,NaN


In [3]:
# Määritä listat; jätä lista tyhjäksi (pelkät hakasulkeet), jos kyseinen tyyppi puuttuu
# Älä kirjoita muuttujan nimeä useampaan kuin yhteen listaan
# Jäljelle jääneet muuttujat tulkitaan määrällisiksi
tarpeettomat = ['nro']
kategoriset = ['sukup', 'perhe', 'koulutus']
mielipideasteikot = ['johto', 'työtov', 'työymp', 'palkkat', 'työteht']
dikotomiset = ['työterv', 'lomaosa', 'kuntosa', 'hieroja']

In [4]:
# Tarpeettomien muuttujien poisto
df = df.drop(tarpeettomat, axis=1)

# Laajennettu kategoristen lista: kategoriset + mielipideasteikot
kategoriset2 = kategoriset + mielipideasteikot

# Lista määrällisistä muuttujista
if kategoriset + dikotomiset:
    df_kvantit = df.drop(kategoriset+dikotomiset, axis=1)
    kvantit = list(df_kvantit.columns)
else:
    kvantit = list(df.columns)

# Excel-työkirja (workbook)
app = xw.App(visible=False)
wb = xw.books.active


# Korrelaatiot                
                
# Korrelaatiot Korrelaatiot-taulukkovälilehdelle
if kvantit:
    ws5 = wb.sheets.add('Korrelaatiot')
    df1 = df_kvantit.corr(min_periods=10)
    ws5.range(1, 1).value = \
        'Määrällisten ja mielipideasteikollisten muuttujien korrelaatiot (n>9)'
    ws5.range(1, 1).font.bold = True
    ws5.range(3, 1).value = df1

    # Korrelaatiot kategoristen muuttujien määräämissä ryhmissä
    if kategoriset:
        rivi = df1.shape[0]+6
        ws5.range(rivi, 1).value = \
            'Korrelaatiot kategoristen muuttujien määräämissä ryhmissä (n>9)'
        ws5.range(rivi, 1).font.bold = True
        for var in kategoriset:
            df1 = df.groupby(var)[kvantit].corr(min_periods=10)
            ws5.range(rivi+2, 1).value = df1
            rivi = rivi + df1.shape[0]+2

            
# Tunnusluvut
            
# Tunnusluvut määrällisille ja mielipideasteikollisille Tunnusluvut-taulukkovälilehdelle
if kvantit:
    ws4 = wb.sheets.add('Tunnusluvut')
    ws4.range(1, 1).value = 'Määrällisten ja mielipideasteikollisten muuttujien tunnusluvut'
    ws4.range(1, 1).font.bold = True
    df1 = df_kvantit.describe()
    ws4.range(3, 1).value = df1
    ws4.range(4, 1).value = 'n'
    ws4.range(5, 1).value = 'keskiarvo'
    ws4.range(6, 1).value = 'keskihajonta'
    ws4.range(7, 1).value = 'pienin'
    ws4.range(11, 1).value = 'suurin'

    # Tunnusluvut kategoristen määräämissä ryhmissä
    if kategoriset2:
        ws4.range(14, 1).value = 'Tunnusluvut kategoristen muuttujien määräämissä ryhmissä'
        ws4.range(14, 1).font.bold = True
        rivi = df1.shape[0] + 8
        for var1 in kategoriset2:
            for var2 in kvantit:
                if var1 != var2:
                    df1 = df.groupby(var1)[var2].describe()
                    df1.index.name = var1
                    ws4.range(rivi, 2).value = var2
                    ws4.range(rivi+1, 1).value = df1
                    ws4.range(rivi+1, 2).value = 'n'
                    ws4.range(rivi+1, 3).value = 'keskiarvo'
                    ws4.range(rivi+1, 4).value = 'keskihajonta'
                    ws4.range(rivi+1, 5).value = 'pienin'
                    ws4.range(rivi+1, 9).value = 'suurin'
                    
                    rivi = rivi + df1.shape[0] + 3

    
# Dikotomiset

if dikotomiset:
    ws3 = wb.sheets.add('Dikotomiset')
    for var in dikotomiset:
        df1 = df[dikotomiset].sum().to_frame()
        df1 = df1.rename(columns = {0:'f'})
        df1['% vastaajista'] = df1['f']/df.shape[0]
        df1.loc['n', '% vastaajista'] = df.shape[0]
        ws3.range(1, 1).value = 'Dikotomisten muuttujien yhteenveto'
        ws3.range(1, 1).font.bold = True
        ws3.range(3, 1).value = df1
        ws3.range((4, 3), (4+len(df1)-2, 3)).number_format = '0,0 %'
                
# Dikotomiset kategoristen määräämissä ryhmissä
    if kategoriset2:
        rivi = df1.shape[0] + 8
        ws3.range(rivi-2, 1).value = \
            'Dikotomiset kategoristen ja mielipideasteikollisten muuttujien määräämissä ryhmissä'
        ws3.range(rivi-2, 1).font.bold = True
        sarake = df1.shape[0]+2
        for var1 in kategoriset2:
            df1 = df.groupby(var1)[dikotomiset].sum()
            ws3.range(rivi, 1).value = df1
            for value in df[var1].dropna().unique():
                value_total = df[var1].value_counts()[value]
                df1.loc[value] = df1.loc[value]/value_total
                df1.loc[value, 'n'] = value_total
                ws3.range(rivi, sarake).value = df1
                ws3.range((rivi+1, sarake+1), (rivi+1+df1.shape[0], 
                    sarake+df1.shape[1]-1)).number_format = '0,0 %'
            rivi = rivi+df1.shape[0]+2
        ws3[:, 1:len(dikotomiset)+1].api.HorizontalAlignment = \
            xw.constants.HAlign.xlHAlignRight

        
# Ristiintaulukoinnit

if len(kategoriset2) > 1:
    rivi = 4
    ws2 = wb.sheets.add('Ristiintaulukoinnit')
    for var1 in kategoriset2:
        for var2 in kategoriset2:
            if var1 != var2:
                df1 = pd.crosstab(df[var1], df[var2], margins=True)
                df2 = pd.crosstab(df[var1], df[var2], normalize='columns')
                df2.loc['n'] = df1.loc['All']
                ws2.range(rivi, 2).value = var2
                ws2.range(rivi, df1.shape[1] + 4).value = var2
                ws2.range(rivi+1, 1).value = df1
                ws2.range(rivi+1, df1.shape[1]+1).value = 'yhteensä'
                ws2.range(rivi+df1.shape[0]+1, 1).value = 'yhteensä'
                ws2.range((rivi+1, df1.shape[1]+3)).value = df2
                ws2.range((rivi+2, df1.shape[1]+4),(rivi+df2.shape[0], 
                    df1.shape[1]+3+df2.shape[1])).number_format = '0,0 %'
                rivi = rivi + df2.shape[0] + 3
    ws2.range(1, 1).value = \
        'Ristiintaulukoinnit kategorisille ja mielipideasteikollisille muuttujille'
    ws2.range(2, 1).value = \
        'Prosentit ovat prosentteja sarakkeen kokonaismäärästä (n)'
    ws2.range((1, 1), (2, 1)).font.bold = True
 

# Frekvenssitaulukot

if kategoriset2:
    # Pidän kirjaa Excelin rivinumerosta rivi-muuttujan avulla
    rivi = 3
    ws1 = wb.sheets.add('Frekvenssitaulukot')
    # Käyn for-silmukalla läpi kaikki kategoriset muuttujat
    for var in kategoriset2:
        # Lasken frekvenssit df1-nimiseen dataframeen
        df1 = pd.crosstab(df[var], 'f')
        # Lasken df1:een prosentit
        df1['%'] = df1/df1.sum()
        # Lisään df1:een Yhteensä-rivin
        df1.loc['yhteensä'] = df1.sum()
        # Frekvenssitaulukko Exceliin
        ws1.range((rivi, 1)).value = df1
        ws1.range((rivi+1, 3),(rivi+len(df1), 3)).number_format = '0,0 %'
        ws1.range((rivi, 2), (rivi, 3)).api.HorizontalAlignment = \
            xw.constants.HAlign.xlHAlignRight
        # Kasvatan rivinumeroa; shape[0] antaa df1:n rivimäärän
        rivi = rivi + df1.shape[0] + 2
    ws1.range(1, 1).value = \
        'Frekvenssitaulukot kategorisille ja mielipideasteikollisille muuttujille'
    ws1.range(1, 1).font.bold = True


# Näytetään Excel
app.visible = True

Lisätietoa Pythonin käytöstä data-analytiikassa https://tilastoapu.wordpress.com/python/